In [2]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
subject = 'Rondônia - Consumo de Cimento (t)'
split_index = 191 #Referente aos 230 anos de input  
train_split = split_index + 1 -30

In [5]:
data = pd.read_csv('2003_mo_model_input_RO.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rondônia - Produção de Cimento (t),Rondônia - IDH,Rondônia - PIB - Estadual,Rondônia - PIB - Construção Civil,Rondônia - PIB - Per Capita,Rondônia - PIB - Preços de Mercado,Rondônia - Desemprego,Rondônia - Consumo de Cimento (t)
0,2003-1,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,4.987000,0.688843,2.068350e+07,1.587672e+06,11.468162,1.828350e+07,8.226062,13.867
1,2003-2,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,4.987000,0.689031,2.071062e+07,1.589518e+06,11.472453,1.829323e+07,8.219943,10.777
2,2003-3,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,4.987000,0.689218,2.073774e+07,1.591364e+06,11.476744,1.830296e+07,8.213823,13.888
3,2003-4,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,4.987000,0.689405,2.076486e+07,1.593210e+06,11.481034,1.831268e+07,8.207703,11.859
4,2003-5,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,4.987000,0.689592,2.079198e+07,1.595056e+06,11.485325,1.832241e+07,8.201584,14.431
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.931530,NaN,NaN,NaN,NaN,NaN,NaN,49.066
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.812259,NaN,NaN,NaN,NaN,NaN,NaN,49.850
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.687985,NaN,NaN,NaN,NaN,NaN,NaN,43.733
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.551024,NaN,NaN,NaN,NaN,NaN,NaN,40.078


In [6]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rondônia - Produção de Cimento (t),Rondônia - IDH,Rondônia - PIB - Estadual,Rondônia - PIB - Construção Civil,Rondônia - PIB - Per Capita,Rondônia - PIB - Preços de Mercado,Rondônia - Desemprego
0,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.115226,-1.859790,-1.684237,0.358666,-2.401614,-1.935575,1.247042
1,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.115226,-1.835964,-1.666360,0.383576,-2.358981,-1.906624,1.233793
2,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.115226,-1.812139,-1.648484,0.408485,-2.316347,-1.877673,1.220544
3,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.115226,-1.788313,-1.630607,0.433395,-2.273714,-1.848722,1.207295
4,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.115226,-1.764487,-1.612730,0.458305,-2.231081,-1.819771,1.194046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,-0.247503,1.280168,1.167829,-1.179294,1.068920,1.031159,-1.264706
188,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,-0.239967,1.265078,1.159325,-1.160375,1.051588,1.020921,-1.291394
189,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,-0.218298,1.249987,1.150821,-1.141456,1.034256,1.010683,-1.318081
190,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,-0.219108,1.234897,1.142317,-1.122538,1.016924,1.000446,-1.344769


In [7]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      13.420
1      11.302
2      16.993
3      16.214
4      15.406
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Rondônia - Consumo de Cimento (t), Length: 240, dtype: float64

In [8]:
# input para treinamento
train_input = input_data.iloc[:train_split]
train_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rondônia - Produção de Cimento (t),Rondônia - IDH,Rondônia - PIB - Estadual,Rondônia - PIB - Construção Civil,Rondônia - PIB - Per Capita,Rondônia - PIB - Preços de Mercado,Rondônia - Desemprego
0,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.115226,-1.859790,-1.684237,0.358666,-2.401614,-1.935575,1.247042
1,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.115226,-1.835964,-1.666360,0.383576,-2.358981,-1.906624,1.233793
2,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.115226,-1.812139,-1.648484,0.408485,-2.316347,-1.877673,1.220544
3,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.115226,-1.788313,-1.630607,0.433395,-2.273714,-1.848722,1.207295
4,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.115226,-1.764487,-1.612730,0.458305,-2.231081,-1.819771,1.194046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,-0.087019,1.457304,1.215933,-1.493457,1.031936,1.145432,-0.079052
158,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,-0.105967,1.457217,1.221174,-1.488264,1.051337,1.152213,-0.144286
159,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,-0.124493,1.457130,1.226414,-1.483070,1.070737,1.158994,-0.209519
160,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,-0.140092,1.457044,1.231655,-1.477877,1.090137,1.165775,-0.274753


In [9]:
# Alvo para treinamento
train_target = target_data.iloc[:train_split]
train_target

0      13.420
1      11.302
2      16.993
3      16.214
4      15.406
        ...  
157    29.662
158    31.408
159    29.484
160    33.601
161    36.905
Name: Rondônia - Consumo de Cimento (t), Length: 162, dtype: float64

In [10]:
#input de test (Ano 2021)
test_input = input_data.iloc[train_split:]
test_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rondônia - Produção de Cimento (t),Rondônia - IDH,Rondônia - PIB - Estadual,Rondônia - PIB - Construção Civil,Rondônia - PIB - Per Capita,Rondônia - PIB - Preços de Mercado,Rondônia - Desemprego
162,-0.601510,-0.653307,0.157879,-1.255326,1.088045,0.763466,-1.213929,-0.154026,1.456870,1.242136,-1.467490,1.128938,1.179337,-0.405220
163,-0.786068,-0.653814,0.548855,-1.118679,1.099377,0.752299,-1.292173,-0.149437,1.456783,1.247376,-1.462297,1.148338,1.186118,-0.470454
164,-0.830387,-0.645656,0.385685,-0.936336,1.110709,0.741131,-1.324219,-0.146620,1.456697,1.252617,-1.457104,1.167739,1.192899,-0.535687
165,-0.801089,-0.634274,0.420965,-0.931790,1.122042,0.729964,-1.344446,-0.144720,1.456610,1.257857,-1.451910,1.187139,1.199680,-0.600921
166,-0.959917,-0.624035,0.548870,-1.168522,1.133374,0.718796,-1.381638,-0.149294,1.456523,1.263098,-1.446717,1.206539,1.206461,-0.666154
167,-1.022309,-0.619791,0.222499,-1.285611,1.144706,0.707629,-1.411208,-0.152432,1.456436,1.268338,-1.441524,1.225940,1.213242,-0.731388
168,-1.074401,-0.620112,-0.614754,-1.446316,1.156038,0.696461,-1.412953,-0.156135,1.456350,1.273579,-1.436331,1.245340,1.220023,-0.796621
169,-1.119597,-0.615973,-0.478229,-1.357832,1.167287,0.681823,-1.491464,-0.163427,1.450470,1.269727,-1.425947,1.240749,1.210256,-0.820061
170,-1.078648,-0.609777,-0.393157,-1.235682,1.178536,0.667184,-1.573805,-0.171276,1.444591,1.265875,-1.415563,1.236157,1.200489,-0.843500
171,-1.055426,-0.602141,-0.557552,-1.065340,1.189784,0.652545,-1.564950,-0.183620,1.438712,1.262023,-1.405179,1.231566,1.190723,-0.866939


In [11]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[train_split:split_index + 1]
test_target

162    36.094
163    42.556
164    41.193
165    37.053
166    33.527
167    25.616
168    25.221
169    23.321
170    23.927
171    24.561
172    20.135
173    34.837
174    34.338
175    39.293
176    41.610
177    43.544
178    35.658
179    29.693
180    32.559
181    24.910
182    28.713
183    32.660
184    31.635
185    35.142
186    36.419
187    40.300
188    35.493
189    38.489
190    35.591
191    27.530
Name: Rondônia - Consumo de Cimento (t), dtype: float64

In [12]:
def validation_splitter(df, div_factor):
    split_factor = len(df)//div_factor
    positions_to_drop = []
    for i in range(split_factor):
        pos = df.shape[0] - (i*div_factor + 1)
        positions_to_drop.append(pos)
    
    return df.drop(positions_to_drop), df.iloc[positions_to_drop]

In [13]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train)
#     display(train_val)
#     display(target)
#     display(target_val)
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val, target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [16]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(25)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [17]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[3692409180, 3905937919, 1989302893, 3197138803, 3724492508, 4063532661, 2308781494, 560018080, 3600029169, 3374916127, 3545162816, 4006485376, 956356329, 3598626078, 3502146001, 283245043, 3160433072, 2194300826, 3603401983, 1835417221, 1739912908, 2483070836, 896320941, 3451515146, 901747655]


Step: 0 ___________________________________________
val_loss: 82.48979949951172
winner_seed: 3692409180


Step: 1 ___________________________________________
val_loss: 66.40487670898438
winner_seed: 3905937919


Step: 2 ___________________________________________
val_loss: 80.51290893554688


Step: 3 ___________________________________________
val_loss: 65.96240997314453
winner_seed: 3197138803


Step: 4 ___________________________________________
val_loss: 79.22322082519531


Step: 5 ___________________________________________
val_loss: 57.249412536621094
winner_seed: 4063532661


Step: 6 ___________________________________________
val_loss: 77.23082733154297


Step: 7 _______________________

In [18]:
trained_model, history = neural_network_model(train_input, 
                                              train_target, 
                                              want_verbose=1, 
                                              seed=winner_seed)

Epoch 1/10000
5/5 [==============================] - 1s 27ms/step - loss: 1573.9688 - val_loss: 2828.5171
Epoch 2/10000
5/5 [==============================] - 0s 7ms/step - loss: 2876.3296 - val_loss: 1504.3385
Epoch 3/10000
5/5 [==============================] - 0s 6ms/step - loss: 1589.5327 - val_loss: 1257.7579
Epoch 4/10000
5/5 [==============================] - 0s 7ms/step - loss: 1405.7231 - val_loss: 580.9241
Epoch 5/10000
5/5 [==============================] - 0s 7ms/step - loss: 677.8060 - val_loss: 279.7362
Epoch 6/10000
5/5 [==============================] - 0s 7ms/step - loss: 251.6516 - val_loss: 413.4127
Epoch 7/10000
5/5 [==============================] - 0s 7ms/step - loss: 295.1518 - val_loss: 282.8040
Epoch 8/10000
5/5 [==============================] - 0s 7ms/step - loss: 233.4630 - val_loss: 297.8383
Epoch 9/10000
5/5 [==============================] - 0s 7ms/step - loss: 241.3016 - val_loss: 268.1328
Epoch 10/10000
5/5 [==============================] - 0s 6ms/step

5/5 [==============================] - 0s 7ms/step - loss: 66.7737 - val_loss: 96.9353
Epoch 81/10000
5/5 [==============================] - 0s 7ms/step - loss: 55.7384 - val_loss: 155.9387
Epoch 82/10000
5/5 [==============================] - 0s 7ms/step - loss: 70.0271 - val_loss: 107.5237
Epoch 83/10000
5/5 [==============================] - 0s 6ms/step - loss: 65.5732 - val_loss: 91.4639
Epoch 84/10000
5/5 [==============================] - 0s 7ms/step - loss: 61.6219 - val_loss: 95.6501
Epoch 85/10000
5/5 [==============================] - 0s 7ms/step - loss: 63.9104 - val_loss: 138.1589
Epoch 86/10000
5/5 [==============================] - 0s 7ms/step - loss: 79.6749 - val_loss: 108.4855
Epoch 87/10000
5/5 [==============================] - 0s 7ms/step - loss: 67.9144 - val_loss: 101.3825
Epoch 88/10000
5/5 [==============================] - 0s 7ms/step - loss: 74.8807 - val_loss: 117.8369
Epoch 89/10000
5/5 [==============================] - 0s 7ms/step - loss: 71.1855 - val_los

5/5 [==============================] - 0s 6ms/step - loss: 50.2391 - val_loss: 96.7932
Epoch 160/10000
5/5 [==============================] - 0s 6ms/step - loss: 53.1639 - val_loss: 95.3901
Epoch 161/10000
5/5 [==============================] - 0s 6ms/step - loss: 53.1707 - val_loss: 99.0603
Epoch 162/10000
5/5 [==============================] - 0s 7ms/step - loss: 55.6297 - val_loss: 104.7408
Epoch 163/10000
5/5 [==============================] - 0s 7ms/step - loss: 51.8778 - val_loss: 108.6546
Epoch 164/10000
5/5 [==============================] - 0s 7ms/step - loss: 61.9241 - val_loss: 100.3735
Epoch 165/10000
5/5 [==============================] - 0s 7ms/step - loss: 52.2080 - val_loss: 91.9602
Epoch 166/10000
5/5 [==============================] - 0s 6ms/step - loss: 55.3053 - val_loss: 98.4127
Epoch 167/10000
5/5 [==============================] - 0s 7ms/step - loss: 52.3158 - val_loss: 99.2846
Epoch 168/10000
5/5 [==============================] - 0s 6ms/step - loss: 55.7845 - v

5/5 [==============================] - 0s 7ms/step - loss: 55.3589 - val_loss: 90.3871
Epoch 239/10000
5/5 [==============================] - 0s 7ms/step - loss: 58.9782 - val_loss: 99.1266
Epoch 240/10000
5/5 [==============================] - 0s 7ms/step - loss: 63.8081 - val_loss: 90.4639
Epoch 241/10000
5/5 [==============================] - 0s 6ms/step - loss: 54.3386 - val_loss: 95.6565
Epoch 242/10000
5/5 [==============================] - 0s 6ms/step - loss: 52.4873 - val_loss: 100.7994
Epoch 243/10000
5/5 [==============================] - 0s 7ms/step - loss: 89.7535 - val_loss: 110.5694
Epoch 244/10000
5/5 [==============================] - 0s 7ms/step - loss: 62.7296 - val_loss: 109.7730
Epoch 245/10000
5/5 [==============================] - 0s 7ms/step - loss: 57.6900 - val_loss: 104.2459
Epoch 246/10000
5/5 [==============================] - 0s 7ms/step - loss: 56.4384 - val_loss: 90.9807
Epoch 247/10000
5/5 [==============================] - 0s 7ms/step - loss: 61.2084 - 

5/5 [==============================] - 0s 7ms/step - loss: 51.4223 - val_loss: 86.1877
Epoch 318/10000
5/5 [==============================] - 0s 6ms/step - loss: 59.8821 - val_loss: 76.7199
Epoch 319/10000
5/5 [==============================] - 0s 7ms/step - loss: 53.0032 - val_loss: 71.9220
Epoch 320/10000
5/5 [==============================] - 0s 7ms/step - loss: 63.9804 - val_loss: 90.0635
Epoch 321/10000
5/5 [==============================] - 0s 7ms/step - loss: 56.1160 - val_loss: 89.0750
Epoch 322/10000
5/5 [==============================] - 0s 7ms/step - loss: 56.5885 - val_loss: 95.4942
Epoch 323/10000
5/5 [==============================] - 0s 8ms/step - loss: 51.6821 - val_loss: 104.9714
Epoch 324/10000
5/5 [==============================] - 0s 6ms/step - loss: 58.5271 - val_loss: 83.3180
Epoch 325/10000
5/5 [==============================] - 0s 7ms/step - loss: 53.6527 - val_loss: 84.9390
Epoch 326/10000
5/5 [==============================] - 0s 7ms/step - loss: 53.1105 - val

...

5/5 [==============================] - 0s 7ms/step - loss: 56.2580 - val_loss: 75.0517
Epoch 792/10000
5/5 [==============================] - 0s 7ms/step - loss: 49.6546 - val_loss: 129.4844
Epoch 793/10000
5/5 [==============================] - 0s 7ms/step - loss: 49.2442 - val_loss: 106.2105
Epoch 794/10000
5/5 [==============================] - 0s 7ms/step - loss: 50.8055 - val_loss: 81.0438
Epoch 795/10000
5/5 [==============================] - 0s 6ms/step - loss: 49.2905 - val_loss: 112.0969
Epoch 796/10000
5/5 [==============================] - 0s 7ms/step - loss: 60.2125 - val_loss: 91.1581
Epoch 797/10000
5/5 [==============================] - 0s 7ms/step - loss: 52.2527 - val_loss: 85.0657
Epoch 798/10000
5/5 [==============================] - 0s 6ms/step - loss: 49.8888 - val_loss: 76.1329
Epoch 799/10000
5/5 [==============================] - 0s 6ms/step - loss: 48.1686 - val_loss: 82.3112
Epoch 800/10000
5/5 [==============================] - 0s 7ms/step - loss: 46.2766 - v

5/5 [==============================] - 0s 7ms/step - loss: 51.4493 - val_loss: 108.8523
Epoch 871/10000
5/5 [==============================] - 0s 7ms/step - loss: 44.4855 - val_loss: 110.9717
Epoch 872/10000
5/5 [==============================] - 0s 6ms/step - loss: 48.3542 - val_loss: 85.8467
Epoch 873/10000
5/5 [==============================] - 0s 6ms/step - loss: 41.7584 - val_loss: 83.3899
Epoch 874/10000
5/5 [==============================] - 0s 7ms/step - loss: 55.9063 - val_loss: 87.6257
Epoch 875/10000
5/5 [==============================] - 0s 7ms/step - loss: 41.8514 - val_loss: 79.9276
Epoch 876/10000
5/5 [==============================] - 0s 8ms/step - loss: 51.4897 - val_loss: 85.9157
Epoch 877/10000
5/5 [==============================] - 0s 8ms/step - loss: 47.3262 - val_loss: 85.0020
Epoch 878/10000
5/5 [==============================] - 0s 7ms/step - loss: 45.2143 - val_loss: 123.2765
Epoch 879/10000
5/5 [==============================] - 0s 6ms/step - loss: 49.5679 - v

5/5 [==============================] - 0s 7ms/step - loss: 44.5576 - val_loss: 90.7014
Epoch 950/10000
5/5 [==============================] - 0s 6ms/step - loss: 45.2484 - val_loss: 89.1992
Epoch 951/10000
5/5 [==============================] - 0s 7ms/step - loss: 43.4228 - val_loss: 100.3694
Epoch 952/10000
5/5 [==============================] - 0s 6ms/step - loss: 43.8047 - val_loss: 90.9069
Epoch 953/10000
5/5 [==============================] - 0s 7ms/step - loss: 48.6692 - val_loss: 103.7689
Epoch 954/10000
5/5 [==============================] - 0s 8ms/step - loss: 43.5416 - val_loss: 93.1486
Epoch 955/10000
5/5 [==============================] - 0s 7ms/step - loss: 43.9520 - val_loss: 93.9493
Epoch 956/10000
5/5 [==============================] - 0s 7ms/step - loss: 41.3931 - val_loss: 110.5351
Epoch 957/10000
5/5 [==============================] - 0s 6ms/step - loss: 56.8952 - val_loss: 252.9694
Epoch 958/10000
5/5 [==============================] - 0s 7ms/step - loss: 83.8738 - 

5/5 [==============================] - 0s 7ms/step - loss: 43.8501 - val_loss: 102.8230
Epoch 1029/10000
5/5 [==============================] - 0s 7ms/step - loss: 52.9084 - val_loss: 101.6067
Epoch 1030/10000
5/5 [==============================] - 0s 7ms/step - loss: 48.3797 - val_loss: 99.4598
Epoch 1031/10000
5/5 [==============================] - 0s 7ms/step - loss: 42.5508 - val_loss: 105.3263
Epoch 1032/10000
5/5 [==============================] - 0s 8ms/step - loss: 48.0644 - val_loss: 94.3879
Epoch 1033/10000
5/5 [==============================] - 0s 7ms/step - loss: 44.1944 - val_loss: 106.2512
Epoch 1034/10000
5/5 [==============================] - 0s 7ms/step - loss: 43.3823 - val_loss: 98.4959
Epoch 1035/10000
5/5 [==============================] - 0s 7ms/step - loss: 49.1410 - val_loss: 138.5345
Epoch 1036/10000
5/5 [==============================] - 0s 7ms/step - loss: 65.0904 - val_loss: 108.1313
Epoch 1037/10000
5/5 [==============================] - 0s 6ms/step - loss:

5/5 [==============================] - 0s 6ms/step - loss: 44.8923 - val_loss: 154.4591
Epoch 1107/10000
5/5 [==============================] - 0s 6ms/step - loss: 71.1632 - val_loss: 112.1601
Epoch 1108/10000
5/5 [==============================] - 0s 6ms/step - loss: 47.4199 - val_loss: 103.0529
Epoch 1109/10000
5/5 [==============================] - 0s 6ms/step - loss: 47.3460 - val_loss: 98.5906
Epoch 1110/10000
5/5 [==============================] - 0s 6ms/step - loss: 47.1373 - val_loss: 98.3456
Epoch 1111/10000
5/5 [==============================] - 0s 6ms/step - loss: 45.3777 - val_loss: 95.5670
Epoch 1112/10000
5/5 [==============================] - 0s 6ms/step - loss: 45.8435 - val_loss: 110.1075
Epoch 1113/10000
5/5 [==============================] - 0s 6ms/step - loss: 48.2216 - val_loss: 99.3980
Epoch 1114/10000
5/5 [==============================] - 0s 6ms/step - loss: 45.0633 - val_loss: 117.3526
Epoch 1115/10000
5/5 [==============================] - 0s 6ms/step - loss: 

In [19]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input.iloc[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [20]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            test_input, 
                                            test_target, 
                                            train_split)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 16ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,32.634468,32.64061,32.6464,32.646896,32.632683,32.618053,32.563469,32.576427,32.592197,32.607864,32.619854,32.645134,32.64209,32.635658,32.641808,32.642918,32.64967,32.656185,32.64912,32.641441,32.66254,32.656422,32.658279,32.662872,32.678341,32.788136,36.389069,39.116642,48.726105,48.732502
Target,36.094,42.556,41.193,37.053,33.527,25.616,25.221,23.321,23.927,24.561,20.135,34.837,34.338,39.293,41.61,43.544,35.658,29.693,32.559,24.91,28.713,32.66,31.635,35.142,36.419,40.3,35.493,38.489,35.591,27.53
Error,3.459534,9.91539,8.5466,4.406105,0.894318,7.002054,7.342468,9.255428,8.665197,8.046864,12.484854,2.191868,1.695911,6.657341,8.968193,10.901081,3.008331,2.963184,0.090122,7.731441,3.949541,0.003578,1.023279,2.479126,3.740658,7.511864,0.896069,0.627644,13.135105,21.202501


In [21]:
display(mae)
display(mape)

5.9598546

0.19694792

In [22]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [23]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             train_split)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 16ms/step
Ano-0: |Prediction[[391.42404]] - Target[368.041]| =  Error: [[23.383057]]; MAPE:[[0.06353384]]
1/1 [==============================] - 0s 16ms/step
Ano-0: |Prediction[[391.79904]] - Target[409.75500000000005]| =  Error: [[17.955963]]; MAPE:[[0.04382122]]
1/1 [==============================] - 0s 19ms/step
Ano-5: |Prediction[[238.43079]] - Target[213.822]| =  Error: [[24.60878]]; MAPE:[[0.11509003]]


[array([[23.383057]], dtype=float32),
 array([[17.955963]], dtype=float32),
 array([[24.60878]], dtype=float32)]

21.9826

0.074148364